In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")

print(f"Weaviate Key:{WEAVIATE_KEY}")
print(f"OpenAI API Key: {OPENAI_API_KEY}")
print(f"OpenAI URL: {OPENAI_URL}")



Weaviate Key:root-user-key
OpenAI API Key: sk-dummy-key-for-local-testing
OpenAI URL: http://host.docker.internal:11434


## Connect to Weaviate

In [2]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY
  }
)

print(client.is_ready())

True


## Create a collection with no vectorizer

In [3]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "MyCollection", and then you need to re-import it again.
from  weaviate.classes.config import Configure, VectorDistances

# Delete the collection if it already exists
if (client.collections.exists("MyCollection")):
    client.collections.delete("MyCollection")

client.collections.create(
    name="MyCollection",
    vector_config=Configure.Vectors.self_provided( # No vectorizer needed
        vector_index_config=Configure.VectorIndex.hnsw( # Optional
        distance_metric=VectorDistances.COSINE # select prefered distance metric 
        )
    ),
)

print(f"Successfully created collection: {'MyCollection'}.")

Successfully created collection: MyCollection.


## Insert an object with a vector

In [4]:
my_collection = client.collections.get("MyCollection")
my_collection.data.insert(
    properties={
        "title": "First Object",
        "foo": 11, 
    },
    vector=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
)

UUID('1316dd90-eb44-4b99-bf40-0c0d9053e482')

In [5]:
res = my_collection.query.fetch_objects(include_vector=True)

print(res.objects[0].properties)
print(res.objects[0].vector)

{'title': 'First Object', 'foo': 11.0}
{'default': [0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612]}


## Insert many objects with their vectors using batch

In [6]:
source = [
    {
        "title": "Second Object",
        "foo": 22,
        "vector": [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
    },
    {
        "title": "Third Object",
        "foo": 33,
        "vector": [0.3, 0.1, -0.1, -0.3, -0.5, -0.7]
    },
    {
        "title": "Fourth Object",
        "foo": 44,
        "vector": [0.4, 0.41, 0.42, 0.43, 0.44, 0.45]
    },
    {
        "title": "Fifth Object",
        "foo": 55,
        "vector": [0.5, 0.5, 0, 0, 0, 0]
    },
]

In [7]:

with my_collection.batch.dynamic() as batch:
    for item in source:
        batch.add_object(
            properties={
                "title": item["title"],
                "foo": item["foo"],
            },
            vector=item["vector"]
        )

## Example with insert_many

In [ ]:
# sample_data = [
#    wc.DataObject(
#       properties={
#          "title": "First Object",
#          "foo": 11, 
#       },
#       vector=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
#    ),
#    wc.DataObject(
#       properties={
#          "title": "Second Object",
#          "foo": 22,
#       },
#       vector=[0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
#    ),
#    wc.DataObject(
#       properties={
#          "title": "Third Object",
#          "foo": 33,
#       },
#       vector=[0.3, 0.1, -0.1, -0.3, -0.5, -0.7]
#    ),
#    wc.DataObject(
#       properties={
#          "title": "Fourth Object",
#         "foo": 44,
#       },
#       vector=[0.4, 0.41, 0.42, 0.43, 0.44, 0.45]
#    ),
#    wc.DataObject(
#       properties={
#          "title": "Fifth Object",
#          "foo": 55,
#       },
#       vector=[0.5, 0.5, 0, 0, 0, 0]
#    ),
# ]

# my_collection.data.insert_many(sample_data)

## Query
Available types of queries you can run when working with vector embeddings (without modules) in **Weaviate**:

1. [near_vector](https://weaviate.io/developers/weaviate/search/similarity#search-with-a-vector)

2. [near_object](https://weaviate.io/developers/weaviate/search/similarity#search-with-an-existing-object)

### nearVector Example
**First example** - Search Weaviate with a vector embedding, and return title property.

See [the docs](https://weaviate.io/developers/weaviate/search/similarity#search-with-a-vector) for more.

In [8]:
response = my_collection.query.near_vector(
    near_vector=[-0.012, 0.021, -0.23, -0.42, 0.5, 0.5],
    limit=2,
)

for item in response.objects:
    print(item.uuid)
    print(item.properties, "\n")

fdb1eb38-0df5-47de-9f0d-dda44b8d7d0a
{'title': 'Second Object', 'foo': 22.0} 

0ff85dd3-d852-40e9-8aae-f8f02ae3c50f
{'title': 'Fourth Object', 'foo': 44.0} 



**Second example** - The same search query, but this time also return `distance`, and `vector`.

In [9]:
from weaviate.classes.query import MetadataQuery

response = my_collection.query.near_vector(
    near_vector=[-0.012, 0.021, -0.23, -0.42, 0.5, 0.5],
    include_vector=True,
    return_metadata=MetadataQuery(distance=True),
    limit=2,
)

for item in response.objects:
    print(item.properties)
    print(item.metadata.distance)
    print(item.vector, "\n")

{'title': 'Second Object', 'foo': 22.0}
0.6506307125091553
{'default': [0.20000000298023224, 0.30000001192092896, 0.4000000059604645, 0.5, 0.6000000238418579, 0.699999988079071]} 

{'title': 'Fourth Object', 'foo': 44.0}
0.8072028756141663
{'default': [0.4000000059604645, 0.4099999964237213, 0.41999998688697815, 0.4300000071525574, 0.4399999976158142, 0.44999998807907104]} 



**Third example** – Same vector query, but this time we will filter on "foo" (which should be greater than 44). Also, let's return "title" and "foo".

See [the docs](https://weaviate.io/developers/weaviate/search/filters#filter-with-one-condition) for more.

In [10]:
from weaviate.classes.query import Filter, MetadataQuery

response = my_collection.query.near_vector(
    near_vector=[-0.012, 0.021, -0.23, -0.42, 0.5, 0.5],
    return_metadata=MetadataQuery(distance=True),
    filters=Filter.by_property("foo").greater_than(30),
    limit=2,
)

for item in response.objects:
    print(item.properties)
    print(item.metadata.distance, "\n")

{'title': 'Fourth Object', 'foo': 44.0}
0.8072028756141663 

{'title': 'Fifth Object', 'foo': 55.0}
0.9925509691238403 



### nearObject Example

Weaviate also allows you to search for similar objects.

See [the docs](https://weaviate.io/developers/weaviate/search/similarity#search-with-an-existing-object) for more.

**Fourth example** - 
Search through `MyCollection` for similar objects, by providing an id from the previous query. 

> Note #1: The id was taken from the query above <br/>
> The generated id for you might be different.

> Note #2: The first object returned is always itself.

In [12]:
from weaviate.classes.query import MetadataQuery

response = my_collection.query.near_object(
    near_object="fdb1eb38-0df5-47de-9f0d-dda44b8d7d0a",
    return_metadata=MetadataQuery(distance=True),
    limit=3,
)

for item in response.objects:
    print(item.uuid)
    print(item.properties)
    print(item.metadata.distance, "\n")

fdb1eb38-0df5-47de-9f0d-dda44b8d7d0a
{'title': 'Second Object', 'foo': 22.0}
0.0 

0ff85dd3-d852-40e9-8aae-f8f02ae3c50f
{'title': 'Fourth Object', 'foo': 44.0}
0.05157363414764404 

1316dd90-eb44-4b99-bf40-0c0d9053e482
{'title': 'First Object', 'foo': 11.0}
0.06506645679473877 



## Close the client

In [13]:
client.close()